In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def tokenizer_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenizer_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize=tokenizer_de,
           init_token='<SOS>',
           eos_token='<EOS>',
           lower=True,
           batch_first=True)
TRG = Field(tokenize=tokenizer_en,
           init_token='<SOS>',
           eos_token='<EOS>',
           lower=True,
           batch_first=True)

train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'),
                                                    fields=(SRC, TRG))
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

train_iter, valid_iter, test_iter = BucketIterator.splits((train_data, valid_data, test_data),
                                                         batch_size=16,
                                                         device=device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_length=100):
        # input_dim, max_length : 인식범위
        # hid dim == d_model
        # pf_dim == d_ff
        super().__init__()
        self.device = device
        self.tok_embedding = nn.Embedding(input_dim, hid_dim) # [len(src vocab), emb dim]
        self.pos_embedding = nn.Embedding(max_length, hid_dim) # 우리 모델은 최대 max_length 만큼의 토큰 개수 만큼을 '한 문장'으로 받아들일 수 있다
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim, dropout, device) for _ in range(n_layers)]) 
        self.droput = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
    def forward(self, src, src_mask):
        # src : [batch size, src time steps]
        # src_mask : [batch size, 1, 1, src time steps]
        batch_size = src.shape[0]
        src_time_steps = src.shape[1]
        
        # batch마다 token index 구해서 token의 위치 값을 만든다
        pos = torch.arange(0, src_time_steps).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # pos : [batch size, src time steps]
        
        # 임베딩 테이블은 차원이 다르지만, 룩업 테이블은 차원이 같다. 그러므로 더할 수 있다.
        src = self.dropout((self.tok_embedding * self.scale) + self.pos_embedding(pos))
        # src : [batch size, src time steps, hid dim]

        for layer in self.layers:
            src = layer(src, src_mask)
        return src # Stacked Encoder의 최종 출력 값

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.feedforward = PositionWiseFeedForward(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
    def forward(self, src, src_mask):
        # src : [batch size, src time steps, hid dim]
        # src_mask : [batch size, 1, 1, src time steps]
        _src, _ = self.self_attention(src, src, src, src_mask) # QKV 계산하기 위해서 src 3개 copy
        src = self.self_attn_layer_norm(src + self.dropout(_src)) # Residual Connection + Norm
        
        _src = self.feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))
        return src

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        assert hid_dim % n_heads == 0
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
    def forward(self, query, key, value, mask=None):
        # query : [batch size, src time steps, hid dim]
        # key : [batch size, src time steps, hid dim]
        # value : [batch size, src time steps, hid dim]
        batch_size = query.shape[0]
        Q = self.fc_q(query) # [batch size, src time steps, hid dim]
        K = self.fc_k(key) 
        V = self.fc_v(value) 
        
        # [batch size, n_heads, src time stpes, head dim]
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale # Attention Score
        # [batch size, n_heads, src time steps, src time steps]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0. -1e10) # padding(0인 부분)에 -무한대 넣기
        
        att_weights = torch.softmax(energy, dim=-1) # [batch size, n_heads, src time steps, src time steps]
        
        att_values = torch.matmul(self.dropout(att_weights), V)
        att_values = att_values.permute(0, 2, 1, 3).contiguous()
        # att_values : [batch size, src time steps, n heads, head dim]
        att_values = att_values.view(batch_size, -1, self.hid_dim) # view로 concat을 대체한다
        # att_values : [batch size, src time steps, hid dim]
        att_values = att_values.fc_o(att_values)
        
        return att_values, att_weights

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        # x : [batch size, src time steps, hid dim]
        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)
        return x